<a href="https://colab.research.google.com/github/Ebenx007/compchem-Compsci-shared-rep/blob/main/data_Engineering_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from pathlib import Path
import sys
import os
import shutil
import tarfile
import zipfile
import subprocess
import pickle
import re
import glob

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/



**DATA CLEANING**


1.   Getting the Data 
2.   Cleaning the Data
3.   Process & Curate Data for use in models







# 1.   GETTING THE DATA

---



**1.1 DATA SOURCE: C/C++ Programming Competition Submissions**



*   Colab is not yet able to download publicly acessable files on another person's google drive.

*   The Programming Competition Submissions are availabe for download [here](https://sites.google.com/site/treebasedcnn/) 





In [ ]:
!cp "/content/drive/My Drive/2020/Fall2020/individual_lab_members/ebenx007/programs.tar.gz" .

In [ ]:
!ls

drive  programs.tar.gz	sample_data




*   VIEW of source code files in tarball of C/C++ programming Competition Submissions 




>   *   TOP LEVEL VIEW  of directories in the dataset tarball.
>  *   Dataset as introduced by  (Mou et 2016), is from 104 online judge programming competition. With 500 C and C++ sobmissions



In [ ]:
#Also abstract into list for easy usage later
submissions_ls = []
submissions_tasks_ls = []
with tarfile.open('programs.tar.gz', 'r') as submissions:
  for member in submissions:
   if member.isdir() and member.name.count ('/') > 0:
     # The '/' > 0 is to ensure the root directory is ignored 
       submissions_tasks_ls.append(member.name)
       print(member.name)
   if member.isfile():
     submissions_ls.append (member.name)
print("\nWorking with {0} source code files as our dataset from coding competition submissions".format(len(submissions_ls)))
print("\nSubmissions are for {0} tasks (think labels)".format(len(submissions_tasks_ls)))
print()

ProgramData/1
ProgramData/2
ProgramData/3
ProgramData/4
ProgramData/5
ProgramData/6
ProgramData/7
ProgramData/8
ProgramData/9
ProgramData/10
ProgramData/11
ProgramData/12
ProgramData/13
ProgramData/14
ProgramData/15
ProgramData/16
ProgramData/17
ProgramData/18
ProgramData/19
ProgramData/20
ProgramData/21
ProgramData/22
ProgramData/23
ProgramData/24
ProgramData/25
ProgramData/26
ProgramData/27
ProgramData/28
ProgramData/29
ProgramData/30
ProgramData/31
ProgramData/32
ProgramData/33
ProgramData/34
ProgramData/35
ProgramData/36
ProgramData/37
ProgramData/38
ProgramData/39
ProgramData/40
ProgramData/41
ProgramData/42
ProgramData/43
ProgramData/44
ProgramData/45
ProgramData/46
ProgramData/47
ProgramData/48
ProgramData/49
ProgramData/50
ProgramData/51
ProgramData/52
ProgramData/53
ProgramData/54
ProgramData/55
ProgramData/56
ProgramData/57
ProgramData/58
ProgramData/59
ProgramData/60
ProgramData/61
ProgramData/62
ProgramData/63
ProgramData/64
ProgramData/65
ProgramData/66
ProgramData/67
Prog

> *   A Look at 2 samples from the 52000 submissions:

In [ ]:
for i in range(2,4):
  print(submissions_ls[i])
  with tarfile.open('programs.tar.gz', 'r') as submissions:
    source_code_file = submissions.extractfile(submissions_ls[i])
    print(source_code_file.read().decode('utf-8'))



ProgramData/1/131.txt
int num(int a,int b,int c)//??????????
{
	int sum=0,j;//sum????
	if(a>c)return 0;//???????????????0
	if(b==1)return 1;//????????????????
	for(j=a;j<=c;j++)//?????????????
		if(c%j==0)sum=sum+num(j,b-1,c/j);
	return sum;
}
int main()
{
	int k,n,m,e=0,i;//e??????
	cin>>n;
	while(e<n)
	{
		cin>>m;
		k=0;//k??m????????????????
		int f=m;
		for(i=2;i<=f;i++)
			if(f%i==0)
			{
				k++;
				f=f/i;
				i=1;
			}
		cout<<num(1,k,m)<<endl;//????????
		e++;
	}
	return 0;
}

ProgramData/1/1076.txt
int decomposition(int m,int k){
    int res=1;
    for(int i=k;i*i<=m;i++){
        if(m%i==0&&m/i>=i){
          
            res+=decomposition(m/i,i);
        }
    }
    return res;
}
int main(){
    int n;
    while(scanf("%d",&n)==1){
        for(int i=0;i<n;i++){
            int m;
            scanf("%d",&m);
            printf("%d\n",decomposition(m,2));
        }
    }
    return 0;
}


**1.2 DATA SOURCE: Juliet C/C++ test suite**

In [ ]:
!wget https://samate.nist.gov/SARD/testsuites/juliet/Juliet_Test_Suite_v1.3_for_C_Cpp.zip

--2020-11-27 02:29:41--  https://samate.nist.gov/SARD/testsuites/juliet/Juliet_Test_Suite_v1.3_for_C_Cpp.zip
Resolving samate.nist.gov (samate.nist.gov)... 129.6.24.116, 2610:20:6005:24::116
Connecting to samate.nist.gov (samate.nist.gov)|129.6.24.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152957342 (146M) [application/zip]
Saving to: ‘Juliet_Test_Suite_v1.3_for_C_Cpp.zip’

Juliet_Test_Suite_v 100%[===================>] 145.87M  45.3MB/s    in 4.2s    

2020-11-27 02:29:46 (34.5 MB/s) - ‘Juliet_Test_Suite_v1.3_for_C_Cpp.zip’ saved [152957342/152957342]



In [ ]:
!ls



*   TOP LEVEL VIEW of Juliet C/C++ test suite dataset. 





>  *   Dataset per NIST (Boland & Black 2017), contains more than 100,000 small test programs in C/C++ , covering 64099 test cases, exhibiting over 100 classes of errors, 118 Common Weakness Enumerations (CWE), for use in testing software assurance tools. 








In [ ]:
#Extract meta data from archive into lists for use later in processing data
juliet_dataset_CWE_testcases_paths_ls = []
juliet_dataset_CWE_ls = []
juliet_dataset_ls = []
with zipfile.ZipFile('Juliet_Test_Suite_v1.3_for_C_Cpp.zip', 'r') as juliet_dataset:
  for member in juliet_dataset.namelist():
    if member.endswith('/') and member.count('/') > 2:
      # > 2  '/' count to ignore none testcases related directories in the archive and focus on your data set of interesti.e. testcases
      juliet_dataset_CWE_testcases_paths_ls.append(member)
      redx = re.findall(r'\bCWE\w.*', member)
      juliet_dataset_CWE_ls.append(redx[0])
      print(redx[0])  
    if (not member.endswith('/')) and member.count('/') > 2:
      # > 2 '/' count to append to list only files in testcases directories, ignoring files inthe root directory  
      juliet_dataset_ls.append(member) 

print("\nWorking with {0} source code files as our dataset from Juliet C/C++ test suite (too large to be printed on colab's strout)".format(len(juliet_dataset_ls)))
print("\nJuliet C/C++ test suite files are divided into {0} CWE IDed testcases (think Labels)".format(len(juliet_dataset_CWE_ls)))
print("\nThe {0} CWE IDed testcases have paths".format(len(juliet_dataset_CWE_testcases_paths_ls)))
print('\n' + str(juliet_dataset_CWE_testcases_paths_ls))
print('\n')

CWE789_Uncontrolled_Mem_Alloc/
CWE789_Uncontrolled_Mem_Alloc/s01/
CWE789_Uncontrolled_Mem_Alloc/s02/
CWE484_Omitted_Break_Statement_in_Switch/
CWE194_Unexpected_Sign_Extension/
CWE194_Unexpected_Sign_Extension/s01/
CWE194_Unexpected_Sign_Extension/s02/
CWE563_Unused_Variable/
CWE588_Attempt_to_Access_Child_of_Non_Structure_Pointer/
CWE78_OS_Command_Injection/
CWE78_OS_Command_Injection/s03/
CWE78_OS_Command_Injection/s09/
CWE78_OS_Command_Injection/s07/
CWE78_OS_Command_Injection/s01/
CWE78_OS_Command_Injection/s05/
CWE78_OS_Command_Injection/s04/
CWE78_OS_Command_Injection/s08/
CWE78_OS_Command_Injection/s06/
CWE78_OS_Command_Injection/s02/
CWE390_Error_Without_Action/
CWE364_Signal_Handler_Race_Condition/
CWE226_Sensitive_Information_Uncleared_Before_Release/
CWE188_Reliance_on_Data_Memory_Layout/
CWE127_Buffer_Underread/
CWE127_Buffer_Underread/s03/
CWE127_Buffer_Underread/s01/
CWE127_Buffer_Underread/s04/
CWE127_Buffer_Underread/s02/
CWE775_Missing_Release_of_File_Descriptor_or_Han

*   Pickling juliet_dataset created lists into a files for use later



In [ ]:
with open('juliet_dataset_CWE_testcases_paths_ls_file', 'wb') as fp:
  pickle.dump (juliet_dataset_CWE_testcases_paths_ls, fp)

In [ ]:
with open('juliet_dataset_ls_file', 'wb') as fp:
  pickle.dump (juliet_dataset_ls, fp)

In [ ]:
with open('juliet_dataset_CWE_ls_file', 'wb') as fp:
  pickle.dump (juliet_dataset_CWE_ls, fp)

In [ ]:
!ls

drive
juliet_dataset_CWE_ls_file
juliet_dataset_CWE_testcases_paths_ls_file
juliet_dataset_ls_file
Juliet_Test_Suite_v1.3_for_C_Cpp.zip
programs.tar.gz
sample_data




*   A Look at 2 samples from the  juliet dataset



In [ ]:
for i in range(100000,100002):
  print(juliet_dataset_ls[i])
  with zipfile.ZipFile('Juliet_Test_Suite_v1.3_for_C_Cpp.zip', 'r') as juliet_dataset:
    juliet_dataset_ls = juliet_dataset.namelist()
    juliet_soure_code_file = juliet_dataset_ls[i]
    print(juliet_dataset.read(juliet_soure_code_file).decode('utf-8'))
 

C/testcases/CWE121_Stack_Based_Buffer_Overflow/s02/CWE121_Stack_Based_Buffer_Overflow__CWE193_char_declare_loop_12.c
/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE121_Stack_Based_Buffer_Overflow__CWE806_char_alloca_ncpy_01.c
Label Definition File: CWE121_Stack_Based_Buffer_Overflow__CWE806.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 121 Stack Based Buffer Overflow
 * BadSource:  Initialize data as a large string
 * GoodSource: Initialize data as a small string
 * Sink: ncpy
 *    BadSink : Copy data to string using strncpy
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#include <wchar.h>

#ifndef OMITBAD

void CWE121_Stack_Based_Buffer_Overflow__CWE806_char_alloca_ncpy_01_bad()
{
    char * data;
    char * dataBuffer = (char *)ALLOCA(100*sizeof(char));
    data = dataBuffer;
    /* FLAW: Initialize data as a large buffer that is larger than the small buffer used in the sink */
    memset(data, 'A', 100-1); /* fill with 'A's */
  

# 2.  CLEANING THE DATA

---
      Graphs from the cleaned data and binaries will be used to generate vectors for ML Models.



**2.1 CLEANING C/C++ Programming Competition Submissions Dataset**

*   REMOVAL FROM CORPUS COMPETITION SUBMISSIONS THAT DO NOT COMPILE ON OUR COLAB (LINUX MACHINE) PLATFORM. ALSO NON UTF8 decocable entries and others
---
---


 >  
1.  Extract contents of Competition submitted txt files from archive
2.  Sort into .cpp and .c code 
3.  Write to files (adding cpp headers for cpp code)
4.  Compile (to test if code is compilable)

    

In [ ]:
with tarfile.open('programs.tar.gz', 'r') as f:
  f.extractall()
  

In [ ]:
!ls

drive
juliet_dataset_CWE_ls_file
juliet_dataset_CWE_testcases_paths_ls_file
juliet_dataset_ls_file
Juliet_Test_Suite_v1.3_for_C_Cpp.zip
ProgramData
programs.tar.gz
sample_data


In [ ]:
active_dir = '/content/ProgramData/'
test_file_counter = 0
for path, subdirs, files in os.walk(active_dir):
    for name in files:
      test_file_counter +=1
print("{} files extracted from Programming Competition Submissions archive. ".format(test_file_counter))

52000 files extracted from Programming Competition Submissions archive. 


In [ ]:
#Variables to hold Meta Data from programming competition dataset 
utf8_decodable_submisisons = []
utf8_undecodable_submissions = []
compiled_cpp_code = []
compiled_c_code = []
uncompilable_utf8_decodable_submissions = []

> * Extract contents of .txt files submitted for Programming Competitions, convert to .cpp or .c files, and compile.

In [ ]:
# 10 Sample to test extraction, conversion and compiling code  
cpp_distinguishing_features = ['.cpp','cin','cout']
for i in range(10):
  with open(os.path.join('', submissions_ls[i]), 'r') as f:
    try:
      file_content = f.read()
    except:
       pass 
    if any(x in file_content for x in cpp_distinguishing_features):
      new_cpp_filename = submissions_ls[i][:-4] + ".cpp" 
      # the -4 for the .txt
      with open(os.path.join('', new_cpp_filename), 'w') as fp:
        cppheader= '''
        #include<iostream> 
        #include <math.h>
        using namespace std;
        '''
        fp.write(cppheader)
        fp.write(file_content)
      subprocess.call(["g++", new_cpp_filename, "-o", submissions_ls[i][:-4] + '.cppout',"-std=c++11", '-w', '-Ofast'])
      print(new_cpp_filename)

    else:
      new_c_filename = submissions_ls[i][:-4] + ".c"
      with open(os.path.join('', new_c_filename), 'w') as fp:
        fp.write(file_content)
      subprocess.call(["gcc", new_c_filename, "-o"+submissions_ls[i][:-4] + '.out',"-std=c99", '-w', '-Ofast'])
      print(new_c_filename)



ProgramData/1/397.cpp
ProgramData/1/147.cpp
ProgramData/1/131.cpp
ProgramData/1/1076.c
ProgramData/1/2272.cpp
ProgramData/1/703.cpp
ProgramData/1/2063.c
ProgramData/1/1892.c
ProgramData/1/889.cpp
ProgramData/1/1342.cpp


In [ ]:
!ls ./ProgramData/1/

1001.txt  131.cppout   1574.txt  1893.txt  2142.txt	388.txt     717.txt
1002.txt  131.txt      1576.txt  1894.txt  2145.txt	393.txt     718.txt
1008.txt  1322.txt     1579.txt  189.txt   214.txt	396.txt     71.txt
1018.txt  1324.txt     1580.txt  18.txt    2151.txt	397.cpp     721.txt
1020.txt  1330.txt     1581.txt  1902.txt  2152.txt	397.cppout  735.txt
1022.txt  1331.txt     1586.txt  1909.txt  2153.txt	397.txt     73.txt
1027.txt  1332.txt     1588.txt  1912.txt  2163.txt	39.txt	    741.txt
1035.txt  1341.txt     1589.txt  1919.txt  2164.txt	400.txt     744.txt
1037.txt  1342.cpp     1593.txt  1923.txt  2172.txt	403.txt     746.txt
1040.txt  1342.cppout  1597.txt  1932.txt  2174.txt	405.txt     753.txt
1042.txt  1342.txt     1604.txt  1935.txt  2182.txt	407.txt     761.txt
1046.txt  1345.txt     1609.txt  1939.txt  2184.txt	408.txt     767.txt
1049.txt  1349.txt     1613.txt  1940.txt  2186.txt	40.txt	    76.txt
104.txt   1353.txt     1616.txt  1944.txt  2195.txt	412.txt     770.tx

In [ ]:
#Extract convert and compile the rest of the submissions
cpp_distinguishing_features = ['.cpp','cin','cout']
print('Compiling the rest of the submitted code ...')
for i in range(len(submissions_ls)):
  with open(os.path.join('', submissions_ls[i]), 'r') as f:
    try:
      file_content = f.read()
      utf8_decodable_submisisons.append(submissions_ls[i])
    except:
       utf8_undecodable_submissions.append(submissions_ls[i])
    if any(x in file_content for x in cpp_distinguishing_features):
      new_cpp_filename = submissions_ls[i][:-4] + ".cpp"
      with open(os.path.join('', new_cpp_filename), 'w') as fp:
        cppheader= '''
        #include<iostream> 
        #include <math.h>
        using namespace std;
        '''
        fp.write(cppheader)
        fp.write(file_content)
      subprocess.call(["g++", new_cpp_filename, "-o", submissions_ls[i][:-4] + '.cppout',"-std=c++11", '-w', '-Ofast'])
      compiled_cpp_code.append(new_cpp_filename)
  

    else:
      new_c_filename = submissions_ls[i][:-4] + ".c"
      with open(os.path.join('', new_c_filename), 'w') as fp:
        fp.write(file_content)
      subprocess.call(["gcc", new_c_filename, "-o"+submissions_ls[i][:-4] + '.out',"-std=c99", '-w', '-Ofast'])
      compiled_c_code.append(new_c_filename)
print('...' )
print('Code compilation completed')

Compiling the rest of the submitted code ...
...
Code compilation completed


In [ ]:
!ls ./ProgramData/

1    104  16  21  27  32  38  43  49  54  6   65  70  76  81  87  92  98
10   11   17  22  28  33  39  44  5   55  60  66  71  77  82  88  93  99
100  12   18  23  29  34  4   45  50  56  61  67  72  78  83  89  94
101  13   19  24  3   35  40  46  51  57  62  68  73  79  84  9   95
102  14   2   25  30  36  41  47  52  58  63  69  74  8   85  90  96
103  15   20  26  31  37  42  48  53  59  64  7   75  80  86  91  97


In [ ]:
!ls ./ProgramData/104

100.c	     1220.cpp	  1437.cpp     180.txt	   519.txt     738.txt
100.out      1220.cppout  1437.cppout  183.c	   521.c       73.c
100.txt      1220.txt	  1437.txt     183.out	   521.out     73.txt
1018.cpp     1221.c	  1438.cpp     183.txt	   521.txt     746.cpp
1018.cppout  1221.out	  1438.cppout  184.c	   522.c       746.cppout
1018.txt     1221.txt	  1438.txt     184.out	   522.out     746.txt
101.c	     1222.c	  1439.cpp     184.txt	   522.txt     74.cpp
101.out      1222.out	  1439.cppout  185.c	   524.c       74.cppout
101.txt      1222.txt	  1439.txt     185.out	   524.out     74.txt
1023.c	     1224.c	  1440.cpp     185.txt	   524.txt     751.cpp
1023.out     1224.out	  1440.cppout  186.c	   525.c       751.txt
1023.txt     1224.txt	  1440.txt     186.out	   525.out     753.cpp
1027.cpp     1229.c	  1446.c       186.txt	   525.txt     753.cppout
1027.cppout  1229.txt	  1446.out     187.c	   527.c       753.txt
1027.txt     122.c	  1446.txt     187.out	   527.out     754.cpp
10

In [ ]:
print("Overview of programming Competition dataset")
print("\n Source code submissions {0}".format(len(submissions_ls)))
print("\n Utf8 Decodable.txt files directories in dataset {0}".format(len(utf8_decodable_submisisons)))
print("\n Utf8 Undecodable .txt files in dataset {0}".format(len(utf8_undecodable_submissions)))
print("\n Compiled .cpp files in dataset {0}".format(len(compiled_cpp_code)))
print("\n Compiled .c files in dataset {0}".format(len(compiled_c_code)))
print("\n Uncompilable utf8 decodable files in dataset {0}".format(len(uncompilable_utf8_decodable_submissions)))

Overview of programming Competition dataset for PHASE 2 of Big Code Data Cleaning Pipeline

 Source code submissions 52000

 Utf8 Decodable.txt files directories in dataset 51752

 Utf8 Undecodable .txt files in dataset 248

 Compiled .cpp files in dataset 15406

 Compiled .c files in dataset 36594

 Uncompilable utf8 decodable files in dataset 0



> *  Pickle Competition Dataset metadata 

In [ ]:
print('Number of Compiled C code files and visual of List')
print(len(compiled_c_code))
print(compiled_c_code)

Number of Compiled C code files and visual of List
36594
['ProgramData/1/1076.c', 'ProgramData/1/2063.c', 'ProgramData/1/1892.c', 'ProgramData/1/1511.c', 'ProgramData/1/26.c', 'ProgramData/1/1170.c', 'ProgramData/1/576.c', 'ProgramData/1/1150.c', 'ProgramData/1/518.c', 'ProgramData/1/1893.c', 'ProgramData/1/1282.c', 'ProgramData/1/1593.c', 'ProgramData/1/2128.c', 'ProgramData/1/900.c', 'ProgramData/1/30.c', 'ProgramData/1/2041.c', 'ProgramData/1/1894.c', 'ProgramData/1/17.c', 'ProgramData/1/1559.c', 'ProgramData/1/1705.c', 'ProgramData/1/836.c', 'ProgramData/1/43.c', 'ProgramData/1/2044.c', 'ProgramData/1/1571.c', 'ProgramData/1/1145.c', 'ProgramData/1/2182.c', 'ProgramData/1/2142.c', 'ProgramData/1/1042.c', 'ProgramData/1/2103.c', 'ProgramData/1/1609.c', 'ProgramData/1/2049.c', 'ProgramData/1/1002.c', 'ProgramData/1/2215.c', 'ProgramData/1/1635.c', 'ProgramData/1/2246.c', 'ProgramData/1/2172.c', 'ProgramData/1/1642.c', 'ProgramData/1/1618.c', 'ProgramData/1/2195.c', 'ProgramData/1/154

In [ ]:
print('Number of Compiled C++ code files and visual of List')
print(len(compiled_cpp_code))
print(compiled_cpp_code)

Number of Compiled C++ code files and visual of List
15406
['ProgramData/1/397.cpp', 'ProgramData/1/147.cpp', 'ProgramData/1/131.cpp', 'ProgramData/1/2272.cpp', 'ProgramData/1/703.cpp', 'ProgramData/1/889.cpp', 'ProgramData/1/1342.cpp', 'ProgramData/1/482.cpp', 'ProgramData/1/820.cpp', 'ProgramData/1/338.cpp', 'ProgramData/1/1393.cpp', 'ProgramData/1/845.cpp', 'ProgramData/1/253.cpp', 'ProgramData/1/146.cpp', 'ProgramData/1/992.cpp', 'ProgramData/1/1521.cpp', 'ProgramData/1/1919.cpp', 'ProgramData/1/150.cpp', 'ProgramData/1/2009.cpp', 'ProgramData/1/1877.cpp', 'ProgramData/1/412.cpp', 'ProgramData/1/1324.cpp', 'ProgramData/1/580.cpp', 'ProgramData/1/1775.cpp', 'ProgramData/1/803.cpp', 'ProgramData/1/413.cpp', 'ProgramData/1/2205.cpp', 'ProgramData/1/1664.cpp', 'ProgramData/1/122.cpp', 'ProgramData/1/1288.cpp', 'ProgramData/1/1446.cpp', 'ProgramData/1/185.cpp', 'ProgramData/1/1360.cpp', 'ProgramData/1/1380.cpp', 'ProgramData/1/954.cpp', 'ProgramData/1/1766.cpp', 'ProgramData/1/665.cpp',

In [ ]:
with open('cpp_compiled_ls_file', 'wb') as fp:
  pickle.dump (compiled_cpp_code, fp)

In [ ]:
with open('submisisons_ls_file', 'wb') as fp:
  pickle.dump (submissions_ls, fp)

In [ ]:
with open('decodable_submisisons_ls_file', 'wb') as fp:
  pickle.dump (utf8_decodable_submisisons, fp)

In [ ]:
with open('c_compiled_ls_file', 'wb') as fp:
  pickle.dump (compiled_c_code, fp)

In [ ]:
!ls

c_compiled_ls_file
cpp_compiled_ls_file
decodable_submisisons_ls_file
drive
juliet_dataset_CWE_ls_file
juliet_dataset_CWE_testcases_paths_ls_file
juliet_dataset_ls_file
Juliet_Test_Suite_v1.3_for_C_Cpp.zip
ProgramData
programs.tar.gz
sample_data
submisisons_ls_file


**2.2 CLEANING: The Juliet C/C++ test suite files** 

*   REMOVAL FROM CORPUS JULIET SOURCE CODE THAT CANNOT COMPILE ON OUR COLAB (LINUX MACHINE) PLATFORM:
---
---


 >  
1.  Extract contents of juliet archive
2.  Create list of source paths  
3.  Create list of paths of Makefiles to ease automation of compilation related tasks. 
4.  Compile to generate binaries

    Graphs from the compiled code and binaries will be used to generate vectors for ML Models.

> *   Extract files from Juliet C/C++ test suite 

In [ ]:
with zipfile.ZipFile('Juliet_Test_Suite_v1.3_for_C_Cpp.zip', 'r') as juliet_dataset:
   juliet_dataset.extractall()
   print('Done Extracting dataset')

Done Extracting dataset


In [ ]:
!ls

C			       juliet_dataset_CWE_testcases_paths_ls_file
c_compiled_ls_file	       juliet_dataset_ls_file
c_json.py		       Juliet_Test_Suite_v1.3_for_C_Cpp.zip
cpp_compiled_ls_file	       ProgramData
decodable_submisisons_ls_file  programs.tar.gz
drive			       sample_data
juliet_dataset_CWE_ls_file     submisisons_ls_file


In [ ]:
#View of extracted Juliet_dataset directory
!ls /content/C

compile_all.bat		 py_common.py
create_per_cwe_files.py  run_analysis_example_tool.py
doc			 testcases
Makefile		 testcasesupport
manifest.xml		 update_main_cpp_and_testcases_h.py


In [ ]:
#Check number of code files i.e. size of dataset
juliet_code_files_dir = '/content/C/testcases/'
test_file_counter = 0
for path, subdirs, files in os.walk(juliet_code_files_dir):
    for name in files:
      test_file_counter +=1
print("{} files in unzipped Juliet archive. Matches our earlier metadata overview with zipFILE module of 106075 source code files".format(test_file_counter))

106075 files in unzipped Juliet archive. Matches our earlier metadata overview with zipFILE module of 106075 source code files


In [ ]:
#Compilating CWE IDed compilable/makeFile-containing CWE IDed directories 
CWE_makefile_dir = []
cwd = os.getcwd()
print("Compiling ...")
for i in range(len(juliet_dataset_ls)):
  if juliet_dataset_ls[i].endswith('Makefile') and 'CWE' in juliet_dataset_ls[i]:
    CWE_makefile_dir.append(juliet_dataset_ls[i])
    redx = re.findall(r'\bCWE\w.*', juliet_dataset_ls[i])
    os.chdir(os.path.dirname(juliet_dataset_ls[i])) 
    print(redx[0][:-8]) 
    #Minus 8 for the length of the word Makefile leading to Makefile containing directories which are the only linun compilables
    with open(os.path.join(cwd, os.path.dirname(juliet_dataset_ls[i]),'compilation.log'), 'w') as lg:
      subprocess.run(["make", "stdout=lg"])
    os.chdir(cwd)

#152 individual CWE IDed dire with Makefiles
print("Compilated all {} Makefile containing CWE IDed directories".format(len(CWE_makefile_dir)))
print('\n')

Compiling ...
CWE789_Uncontrolled_Mem_Alloc/s01/
CWE789_Uncontrolled_Mem_Alloc/s02/
CWE484_Omitted_Break_Statement_in_Switch/
CWE194_Unexpected_Sign_Extension/s01/
CWE194_Unexpected_Sign_Extension/s02/
CWE563_Unused_Variable/
CWE588_Attempt_to_Access_Child_of_Non_Structure_Pointer/
CWE78_OS_Command_Injection/s03/
CWE78_OS_Command_Injection/s01/
CWE78_OS_Command_Injection/s04/
CWE78_OS_Command_Injection/s02/
CWE390_Error_Without_Action/
CWE364_Signal_Handler_Race_Condition/
CWE188_Reliance_on_Data_Memory_Layout/
CWE127_Buffer_Underread/s03/
CWE127_Buffer_Underread/s01/
CWE127_Buffer_Underread/s02/
CWE775_Missing_Release_of_File_Descriptor_or_Handle/
CWE667_Improper_Locking/
CWE843_Type_Confusion/
CWE587_Assignment_of_Fixed_Address_to_Pointer/
CWE480_Use_of_Incorrect_Operator/
CWE762_Mismatched_Memory_Management_Routines/s03/
CWE762_Mismatched_Memory_Management_Routines/s07/
CWE762_Mismatched_Memory_Management_Routines/s01/
CWE762_Mismatched_Memory_Management_Routines/s05/
CWE762_Mismatc

In [ ]:
print('Number of CWE IDed Makefiles and visualize in a List')
print(len(CWE_makefile_dir))
print(CWE_makefile_dir)

Number of CWE IDed Makefiles and visualize in a List
152
['C/testcases/CWE789_Uncontrolled_Mem_Alloc/s01/Makefile', 'C/testcases/CWE789_Uncontrolled_Mem_Alloc/s02/Makefile', 'C/testcases/CWE484_Omitted_Break_Statement_in_Switch/Makefile', 'C/testcases/CWE194_Unexpected_Sign_Extension/s01/Makefile', 'C/testcases/CWE194_Unexpected_Sign_Extension/s02/Makefile', 'C/testcases/CWE563_Unused_Variable/Makefile', 'C/testcases/CWE588_Attempt_to_Access_Child_of_Non_Structure_Pointer/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s03/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s01/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s04/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s02/Makefile', 'C/testcases/CWE390_Error_Without_Action/Makefile', 'C/testcases/CWE364_Signal_Handler_Race_Condition/Makefile', 'C/testcases/CWE188_Reliance_on_Data_Memory_Layout/Makefile', 'C/testcases/CWE127_Buffer_Underread/s03/Makefile', 'C/testcases/CWE127_Buffer_Underread/s01/Makefile', 'C/testcases/

In [ ]:
with open('CWE_makefile_dir_ls_file', 'wb') as fp:
  pickle.dump (CWE_makefile_dir, fp)

In [ ]:
!ls

C
c_compiled_ls_file
c_json.py
cpp_compiled_ls_file
CWE_makefile_dir_ls_file
decodable_submisisons_ls_file
drive
juliet_dataset_CWE_ls_file
juliet_dataset_CWE_testcases_paths_ls_file
juliet_dataset_ls_file
Juliet_Test_Suite_v1.3_for_C_Cpp.zip
phase1_programing_competition.tar
ProgramData
programming_competition_c_ast_jsons_ls_file
programs.tar.gz
sample_data
submisisons_ls_file


# 3.   PROCESS & CURATE DATA FOR USE IN MODELS

---



*   GENERATE GRAPHS FROM COMPILED CODE & BINARIES.
---
---


 >  * 
1.   Use pycparser to extract ASTs from C code and serialize C ASTs graphs 
2.   Serialize to Json files
3.   Use Angr to extract CFG from C binaries
4.   Archive processed data for use in Vector generation for ML models 



    Store Json graphs in Googledrive for use in generating vectors for the ML Models.


**3.1 Using pycparser and Json serializing script to generate serialized ASTs from compilable Competition C code submissions** 
----

> *   pycparser and Json serializing script for single file C programs submitted in programming competition 

In [ ]:
%%writefile c_json.py

# Original script from:
#-----------------------------------------------------------------
# pycparser: serialize_ast.py
#
# Simple example of serializing AST
#
# Hart Chu [https://github.com/CtheSky]
# Eli Bendersky [https://eli.thegreenplace.net/]
# License: BSD
#-----------------------------------------------------------------
## Adjusted for use here by Valentine Eben.

import json
import sys
import re



from pycparser import parse_file, c_ast
from pycparser.plyparser import Coord


RE_CHILD_ARRAY = re.compile(r'(.*)\[(.*)\]')
RE_INTERNAL_ATTR = re.compile('__.*__')


class CJsonError(Exception):
    pass


def memodict(fn):
    """ Fast memoization decorator for a function taking a single argument """
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = fn(key)
            return ret
    return memodict().__getitem__


@memodict
def child_attrs_of(klass):
    """
    Given a Node class, get a set of child attrs.
    Memoized to avoid highly repetitive string manipulation
    """
    non_child_attrs = set(klass.attr_names)
    all_attrs = set([i for i in klass.__slots__ if not RE_INTERNAL_ATTR.match(i)])
    return all_attrs - non_child_attrs


def to_dict(node):
    """ Recursively convert an ast into dict representation. """
    klass = node.__class__

    result = {}

    # Metadata
    result['_nodetype'] = klass.__name__

    # Local node attributes
    for attr in klass.attr_names:
        result[attr] = getattr(node, attr)

    # Coord object
    if node.coord:
        result['coord'] = str(node.coord)
    else:
        result['coord'] = None

    # Child attributes
    for child_name, child in node.children():
        # Child strings are either simple (e.g. 'value') or arrays (e.g. 'block_items[1]')
        match = RE_CHILD_ARRAY.match(child_name)
        if match:
            array_name, array_index = match.groups()
            array_index = int(array_index)
            # arrays come in order, so we verify and append.
            result[array_name] = result.get(array_name, [])
            if array_index != len(result[array_name]):
                raise CJsonError('Internal ast error. Array {} out of order. '
                    'Expected index {}, got {}'.format(
                    array_name, len(result[array_name]), array_index))
            result[array_name].append(to_dict(child))
        else:
            result[child_name] = to_dict(child)

    # Any child attributes that were missing need "None" values in the json.
    for child_attr in child_attrs_of(klass):
        if child_attr not in result:
            result[child_attr] = None

    return result


def to_json(node, **kwargs):
    """ Convert ast node to json string """
    return json.dumps(to_dict(node), **kwargs)


def file_to_dict(filename):
    """ Load C file into dict representation of ast """
    ## ast = parse_file(filename, use_cpp=True) my addition of pycparser fakehead to allow for intraction with files
    ast = parse_file(filename, use_cpp=True,
            cpp_path='gcc',
            cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return to_dict(ast)


def file_to_json(filename, **kwargs):
    """ Load C file into json string representation of ast """
     ## ast = parse_file(filename, use_cpp=True) my addition of pycparser fakehead to allow for intraction with files
    ast = parse_file(filename, use_cpp=True,
            cpp_path='gcc',
            cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return to_json(ast, **kwargs)


def _parse_coord(coord_str):
    """ Parse coord string (file:line[:column]) into Coord object. """
    if coord_str is None:
        return None

    vals = coord_str.split(':')
    vals.extend([None] * 3)
    filename, line, column = vals[:3]
    return Coord(filename, line, column)


def _convert_to_obj(value):
    """
    Convert an object in the dict representation into an object.
    Note: Mutually recursive with from_dict.
    """
    value_type = type(value)
    if value_type == dict:
        return from_dict(value)
    elif value_type == list:
        return [_convert_to_obj(item) for item in value]
    else:
        # String
        return value


def from_dict(node_dict):
    """ Recursively build an ast from dict representation """
    class_name = node_dict.pop('_nodetype')

    klass = getattr(c_ast, class_name)

    # Create a new dict containing the key-value pairs which we can pass
    # to node constructors.
    objs = {}
    for key, value in node_dict.items():
        if key == 'coord':
            objs[key] = _parse_coord(value)
        else:
            objs[key] = _convert_to_obj(value)

    # Use keyword parameters, which works thanks to beautifully consistent
    # ast Node initializers.
    return klass(**objs)


def from_json(ast_json):
    """ Build an ast from json string representation """
    return from_dict(json.loads(ast_json))


#------------------------------------------------------------------------------
if __name__ == "__main__":
    if len(sys.argv) > 1:
        # Some test code...
        # Do trip from C -> ast -> dict -> ast -> json, then print.
        ast_dict = file_to_dict(sys.argv[1])
        ast = from_dict(ast_dict)
        json_object = to_json(ast, sort_keys=True, indent=4)
        print(json_object)
        with open(sys.argv[1][:-2]+ ".json", "w") as outfile:
          outfile.write(json_object)
        ## little adjustment to save to a file names almost as input file. The minus 2 for ".c"
    else:
        print("Please provide a filename as argument")

Writing c_json.py


In [ ]:
!ls

C			       juliet_dataset_CWE_testcases_paths_ls_file
c_compiled_ls_file	       juliet_dataset_ls_file
c_json.py		       Juliet_Test_Suite_v1.3_for_C_Cpp.zip
cpp_compiled_ls_file	       ProgramData
decodable_submisisons_ls_file  programs.tar.gz
drive			       sample_data
juliet_dataset_CWE_ls_file     submisisons_ls_file


In [ ]:
#Test AST generating script 
!python3 c_json.py /content/ProgramData/1/1076.c

{
    "_nodetype": "FileAST",
    "coord": null,
    "ext": [
        {
            "_nodetype": "FuncDef",
            "body": {
                "_nodetype": "Compound",
                "block_items": [
                    {
                        "_nodetype": "Decl",
                        "bitsize": null,
                        "coord": "/content/ProgramData/1/1076.c:2:9",
                        "funcspec": [],
                        "init": {
                            "_nodetype": "Constant",
                            "coord": "/content/ProgramData/1/1076.c:2:13",
                            "type": "int",
                            "value": "1"
                        },
                        "name": "res",
                        "quals": [],
                        "storage": [],
                        "type": {
                            "_nodetype": "TypeDecl",
                            "coord": "/content/ProgramData/1/1076.c:2:9",
                            "

In [ ]:
#Check that json ast (1076.json) was created in addtion to the stdout priprint 
!ls ./ProgramData/1/

1001.c	     1362.cpp	  1671.cpp     2037.txt     257.txt	631.cppout
1001.out     1362.cppout  1671.cppout  2040.c	    259.cpp	631.txt
1001.txt     1362.txt	  1671.txt     2040.out     259.cppout	664.cpp
1002.c	     1376.c	  1679.cpp     2040.txt     259.txt	664.cppout
1002.out     1376.out	  1679.cppout  2041.c	    25.c	664.txt
1002.txt     1376.txt	  1679.txt     2041.out     25.out	665.cpp
1008.c	     1379.cpp	  1681.cpp     2041.txt     25.txt	665.cppout
1008.out     1379.cppout  1681.cppout  2044.c	    261.cpp	665.txt
1008.txt     1379.txt	  1681.txt     2044.out     261.cppout	666.cpp
1018.c	     137.cpp	  1683.cpp     2044.txt     261.txt	666.cppout
1018.out     137.cppout   1683.cppout  2047.c	    26.c	666.txt
1018.txt     137.txt	  1683.txt     2047.out     26.out	66.c
1020.cpp     1380.cpp	  1687.c       2047.txt     26.txt	66.out
1020.cppout  1380.cppout  1687.out     2049.c	    270.cpp	66.txt
1020.txt     1380.txt	  1687.txt     2049.out     270.cppout	671.cpp
1022.cpp     1

In [ ]:
!ls

C			       juliet_dataset_CWE_testcases_paths_ls_file
c_compiled_ls_file	       juliet_dataset_ls_file
c_json.py		       Juliet_Test_Suite_v1.3_for_C_Cpp.zip
cpp_compiled_ls_file	       ProgramData
decodable_submisisons_ls_file  programs.tar.gz
drive			       sample_data
juliet_dataset_CWE_ls_file     submisisons_ls_file


In [ ]:
#Generate the rest of the json ASTs from the Programming Competition submissions 
for i in range(len(compiled_c_code)):
    subprocess.run(["python3", "c_json.py", compiled_c_code[i]])
print('Done generating json ASTs. Verify with ls cmd')  

Done generating json ASTs.  Use list files to verify success


In [ ]:
#Verify success of json generating script
programming_competition_c_ast_jsons = []
paths = Path('./ProgramData').glob('**/*.json')
for path in paths:
  programming_competition_c_ast_jsons.append(str(path))
  # str because path is an object not string
print("Generated {} jsons of C ASTs from the programming submissions".format(len(programming_competition_c_ast_jsons)))
print(programming_competition_c_ast_jsons)
print('\n')

Generated 36594 jsons of C ASTs from the programming submissions
['ProgramData/16/401.json', 'ProgramData/16/1107.json', 'ProgramData/16/11.json', 'ProgramData/16/1350.json', 'ProgramData/16/1461.json', 'ProgramData/16/581.json', 'ProgramData/16/1381.json', 'ProgramData/16/625.json', 'ProgramData/16/1067.json', 'ProgramData/16/830.json', 'ProgramData/16/1033.json', 'ProgramData/16/1087.json', 'ProgramData/16/714.json', 'ProgramData/16/283.json', 'ProgramData/16/638.json', 'ProgramData/16/1204.json', 'ProgramData/16/841.json', 'ProgramData/16/1188.json', 'ProgramData/16/212.json', 'ProgramData/16/272.json', 'ProgramData/16/275.json', 'ProgramData/16/856.json', 'ProgramData/16/5.json', 'ProgramData/16/45.json', 'ProgramData/16/145.json', 'ProgramData/16/1446.json', 'ProgramData/16/146.json', 'ProgramData/16/676.json', 'ProgramData/16/165.json', 'ProgramData/16/645.json', 'ProgramData/16/629.json', 'ProgramData/16/224.json', 'ProgramData/16/448.json', 'ProgramData/16/990.json', 'ProgramDa

In [ ]:
#check for json ASTs in  ProgramData subdirectories e.g. ProgramData/1/
!ls ./ProgramData/1/ 

1001.c	     1376.c	  1653.json    2023.txt     2278.txt	 606.txt
1001.json    1376.json	  1653.out     2037.c	    227.cpp	 608.c
1001.out     1376.out	  1653.txt     2037.json    227.cppout	 608.json
1001.txt     1376.txt	  1656.c       2037.out     227.txt	 608.out
1002.c	     1379.cpp	  1656.json    2037.txt     2290.cpp	 608.txt
1002.json    1379.cppout  1656.out     2040.c	    2290.cppout  609.c
1002.out     1379.txt	  1656.txt     2040.json    2290.txt	 609.json
1002.txt     137.cpp	  1664.cpp     2040.out     230.cpp	 609.out
1008.c	     137.cppout   1664.cppout  2040.txt     230.cppout	 609.txt
1008.json    137.txt	  1664.txt     2041.c	    230.txt	 621.c
1008.out     1380.cpp	  1671.cpp     2041.json    234.cpp	 621.json
1008.txt     1380.cppout  1671.cppout  2041.out     234.cppout	 621.out
1018.c	     1380.txt	  1671.txt     2041.txt     234.txt	 621.txt
1018.json    1381.cpp	  1679.cpp     2044.c	    241.cpp	 622.cpp
1018.out     1381.cppout  1679.cppout  2044.json    241.cp

In [ ]:
with open('programming_competition_c_ast_jsons_ls_file', 'wb') as fp:
  pickle.dump (programming_competition_c_ast_jsons, fp)

**3.2 Using pycparser and Json serializing script to generate serialized ASTs from juliet C code**
---- 

In [ ]:
##################### copy partially processed files for further processing ########### 
!cp "/content/drive/My Drive/2020/Fall2020/individual_lab_members/ebenx007/phase1_processed_juliet_dataset.zip" . 

In [ ]:
!ls

C  drive  phase1_processed_juliet_dataset.zip  sample_data


In [ ]:
with zipfile.ZipFile('phase1_processed_juliet_dataset.zip', 'r') as p_juliet_dataset:
   p_juliet_dataset.extractall()
   print('Done Extracting processed juliet dataset')

Done Extracting processed juliet dataset


In [ ]:
!ls

C  drive  phase1_processed_juliet_dataset.zip  sample_data


In [ ]:
#cd cmd to directoy where files where saved before archived
%cd C

/content/C


In [ ]:
#loading pickled data back CWE_makefile_dir
CWE_makefile_dir = []
with open("CWE_makefile_dir_ls_file", "rb") as fld:
  CWE_makefile_dir = pickle.load(fld)

In [ ]:
#change back to home dir
%cd ..

/content


In [ ]:
print(len(CWE_makefile_dir))
print(CWE_makefile_dir)

152
['C/testcases/CWE789_Uncontrolled_Mem_Alloc/s01/Makefile', 'C/testcases/CWE789_Uncontrolled_Mem_Alloc/s02/Makefile', 'C/testcases/CWE484_Omitted_Break_Statement_in_Switch/Makefile', 'C/testcases/CWE194_Unexpected_Sign_Extension/s01/Makefile', 'C/testcases/CWE194_Unexpected_Sign_Extension/s02/Makefile', 'C/testcases/CWE563_Unused_Variable/Makefile', 'C/testcases/CWE588_Attempt_to_Access_Child_of_Non_Structure_Pointer/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s03/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s01/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s04/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s02/Makefile', 'C/testcases/CWE390_Error_Without_Action/Makefile', 'C/testcases/CWE364_Signal_Handler_Race_Condition/Makefile', 'C/testcases/CWE188_Reliance_on_Data_Memory_Layout/Makefile', 'C/testcases/CWE127_Buffer_Underread/s03/Makefile', 'C/testcases/CWE127_Buffer_Underread/s01/Makefile', 'C/testcases/CWE127_Buffer_Underread/s02/Makefile', 'C/testcases/C

> *   pycparser and Json serializing script for multi file programs 

In [ ]:
#Start by cloneing pycparser to ease access to pycparser's Fakeheaders, needed for AST generation, instead of trying to access those that come with the pycpaser package. Eli's idea and it works  
!git clone  https://github.com/eliben/pycparser.git 


Cloning into 'pycparser'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 2548 (delta 25), reused 38 (delta 13), pack-reused 2489
Receiving objects: 100% (2548/2548), 1.15 MiB | 2.90 MiB/s, done.
Resolving deltas: 100% (1702/1702), done.


In [ ]:
%%writefile multi_file_c_json.py

# Original script from:
#-----------------------------------------------------------------
# pycparser: serialize_ast.py
#
# Simple example of serializing AST
#
# Hart Chu [https://github.com/CtheSky]
# Eli Bendersky [https://eli.thegreenplace.net/]
# License: BSD
#-----------------------------------------------------------------
## Adjusted for use here by Valentine Eben notice the addtion of the headers for this specific juliet dataset as '-Icontent/C'.

import json
import sys
import re


from pycparser import parse_file, c_ast
from pycparser.plyparser import Coord


RE_CHILD_ARRAY = re.compile(r'(.*)\[(.*)\]')
RE_INTERNAL_ATTR = re.compile('__.*__')


class CJsonError(Exception):
    pass


def memodict(fn):
    """ Fast memoization decorator for a function taking a single argument """
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = fn(key)
            return ret
    return memodict().__getitem__


@memodict
def child_attrs_of(klass):
    """
    Given a Node class, get a set of child attrs.
    Memoized to avoid highly repetitive string manipulation
    """
    non_child_attrs = set(klass.attr_names)
    all_attrs = set([i for i in klass.__slots__ if not RE_INTERNAL_ATTR.match(i)])
    return all_attrs - non_child_attrs


def to_dict(node):
    """ Recursively convert an ast into dict representation. """
    klass = node.__class__

    result = {}

    # Metadata
    result['_nodetype'] = klass.__name__

    # Local node attributes
    for attr in klass.attr_names:
        result[attr] = getattr(node, attr)

    # Coord object
    if node.coord:
        result['coord'] = str(node.coord)
    else:
        result['coord'] = None

    # Child attributes
    for child_name, child in node.children():
        # Child strings are either simple (e.g. 'value') or arrays (e.g. 'block_items[1]')
        match = RE_CHILD_ARRAY.match(child_name)
        if match:
            array_name, array_index = match.groups()
            array_index = int(array_index)
            # arrays come in order, so we verify and append.
            result[array_name] = result.get(array_name, [])
            if array_index != len(result[array_name]):
                raise CJsonError('Internal ast error. Array {} out of order. '
                    'Expected index {}, got {}'.format(
                    array_name, len(result[array_name]), array_index))
            result[array_name].append(to_dict(child))
        else:
            result[child_name] = to_dict(child)

    # Any child attributes that were missing need "None" values in the json.
    for child_attr in child_attrs_of(klass):
        if child_attr not in result:
            result[child_attr] = None

    return result


def to_json(node, **kwargs):
    """ Convert ast node to json string """
    return json.dumps(to_dict(node), **kwargs)


def file_to_dict(filename):
    """ Load C file into dict representation of ast """
     ## ast = parse_file(filename, use_cpp=True) my addition of pycparser fakehead to allow for intraction with files.
    ast = parse_file(filename, use_cpp=True,
            cpp_path='gcc',
            cpp_args=['-nostdinc','-E','-IC/testcasesupport/', r'-Ipycparser/utils/fake_libc_include'])
    ## also -nostdinc to supress hard coded system headers creating issues and adding the std for this project 
    return to_dict(ast)


def file_to_json(filename, **kwargs):
    """ Load C file into json string representation of ast """
     ## ast = parse_file(filename, use_cpp=True) my addition of pycparser fakehead to allow for intraction with files.
    ast = parse_file(filename, use_cpp=True,
            cpp_path='gcc',
            cpp_args=['-nostdinc','-E','-IC/testcasesupport/', r'-Ipycparser/utils/fake_libc_include'])
   ## also -nostdinc to supress hard coded system headers creating issues
    return to_json(ast, **kwargs)


def _parse_coord(coord_str):
    """ Parse coord string (file:line[:column]) into Coord object. """
    if coord_str is None:
        return None

    vals = coord_str.split(':')
    vals.extend([None] * 3)
    filename, line, column = vals[:3]
    return Coord(filename, line, column)


def _convert_to_obj(value):
    """
    Convert an object in the dict representation into an object.
    Note: Mutually recursive with from_dict.
    """
    value_type = type(value)
    if value_type == dict:
        return from_dict(value)
    elif value_type == list:
        return [_convert_to_obj(item) for item in value]
    else:
        # String
        return value


def from_dict(node_dict):
    """ Recursively build an ast from dict representation """
    class_name = node_dict.pop('_nodetype')

    klass = getattr(c_ast, class_name)

    # Create a new dict containing the key-value pairs which we can pass
    # to node constructors.
    objs = {}
    for key, value in node_dict.items():
        if key == 'coord':
            objs[key] = _parse_coord(value)
        else:
            objs[key] = _convert_to_obj(value)

    # Use keyword parameters, which works thanks to beautifully consistent
    # ast Node initializers.
    return klass(**objs)


def from_json(ast_json):
    """ Build an ast from json string representation """
    return from_dict(json.loads(ast_json))


#------------------------------------------------------------------------------
if __name__ == "__main__":
    if len(sys.argv) > 1:
        # Some test code...
        # Do trip from C -> ast -> dict -> ast -> json, then print.
        ast_dict = file_to_dict(sys.argv[1])
        ast = from_dict(ast_dict)
        json_object = to_json(ast, sort_keys=True, indent=4)
        print(json_object)
        with open(sys.argv[1][:-2]+ ".json", "w") as outfile:
          outfile.write(json_object)
        ## little adjustment to save to a file names almost as input file. The minus 2 for ".c"
    else:
        print("Please provide a filename as argument")


Writing multi_file_c_json.py


In [ ]:
#check script was created
!ls

C      multi_file_c_json.py		    pycparser
drive  phase1_processed_juliet_dataset.zip  sample_data


In [ ]:
#Test multiple file C program Json Genrating script
!python3 multi_file_c_json.py C/testcases/CWE675_Duplicate_Operations_on_Resource/CWE675_Duplicate_Operations_on_Resource__fopen_52a.c

{
    "_nodetype": "FileAST",
    "coord": null,
    "ext": [
        {
            "_nodetype": "Typedef",
            "coord": "pycparser/utils/fake_libc_include/_fake_typedefs.h:4:13",
            "name": "size_t",
            "quals": [],
            "storage": [
                "typedef"
            ],
            "type": {
                "_nodetype": "TypeDecl",
                "coord": "pycparser/utils/fake_libc_include/_fake_typedefs.h:4:13",
                "declname": "size_t",
                "quals": [],
                "type": {
                    "_nodetype": "IdentifierType",
                    "coord": "pycparser/utils/fake_libc_include/_fake_typedefs.h:4:9",
                    "names": [
                        "int"
                    ]
                }
            }
        },
        {
            "_nodetype": "Typedef",
            "coord": "pycparser/utils/fake_libc_include/_fake_typedefs.h:5:13",
            "name": "__builtin_va_list",
            "quals":

In [ ]:
#cd to check creation of json AST (C/testcases/CWE675_Duplicate_Operations_on_Resource/CWE675_Duplicate_Operations_on_Resource__fopen_52a.json )
%cd C/testcases/CWE675_Duplicate_Operations_on_Resource/

/content/C/testcases/CWE675_Duplicate_Operations_on_Resource


In [ ]:
!ls

compilation.log
CWE675.bat
CWE675_Duplicate_Operations_on_Resource__fopen_01.c
CWE675_Duplicate_Operations_on_Resource__fopen_01.o
CWE675_Duplicate_Operations_on_Resource__fopen_02.c
CWE675_Duplicate_Operations_on_Resource__fopen_02.o
CWE675_Duplicate_Operations_on_Resource__fopen_03.c
CWE675_Duplicate_Operations_on_Resource__fopen_03.o
CWE675_Duplicate_Operations_on_Resource__fopen_04.c
CWE675_Duplicate_Operations_on_Resource__fopen_04.o
CWE675_Duplicate_Operations_on_Resource__fopen_05.c
CWE675_Duplicate_Operations_on_Resource__fopen_05.o
CWE675_Duplicate_Operations_on_Resource__fopen_06.c
CWE675_Duplicate_Operations_on_Resource__fopen_06.o
CWE675_Duplicate_Operations_on_Resource__fopen_07.c
CWE675_Duplicate_Operations_on_Resource__fopen_07.o
CWE675_Duplicate_Operations_on_Resource__fopen_08.c
CWE675_Duplicate_Operations_on_Resource__fopen_08.o
CWE675_Duplicate_Operations_on_Resource__fopen_09.c
CWE675_Duplicate_Operations_on_Resource__fopen_09.o
CWE675_Duplicate_Operations_on_Resour

In [ ]:
#back to home dir from which path begins
%cd ..

/content


In [ ]:
#Create list of all compiled juliet code along with c and cpp compiled code for easy parsing with appropriate tools


compilable_juliet_code = []
compilable_juliet_c_code = []
compilable_juliet_cpp_code = []
paths = Path('./C/testcases').glob('**/*.o')
for path in paths:
  if re.search(r'CWE',str(path.name)):
    #.name to make sure the last part of the name i.e. before the suffix is verified
    compilable_juliet_code.append(str(path))
    if Path(str(path)[:-2]+'.c').exists():
      #Back to Path allows you to use the .exists atribute, while str allows -2 for ".o" and + operator for +'.c'
      compilable_juliet_c_code.append(str(path)[:-2]+'.c')
    if Path(str(path)[:-2]+'.cpp').exists():
      compilable_juliet_cpp_code.append(str(path)[:-2]+'.cpp')
 
  # str because path is an object not string
print("Total of {} compilable code:".format(len(compilable_juliet_code)))
print(compilable_juliet_code[0])
print("\n{} of the compilable Juliet code is C:".format(len(compilable_juliet_c_code)))
print(compilable_juliet_c_code[0])
print("\n{} of the compilable Juliet code is C++:".format(len(compilable_juliet_cpp_code)))
print(compilable_juliet_cpp_code[0])
print('\n35551 + 30441 = {} matching the total compilable code in the Juliet dataset'.format(35551 + 30441))
print('\n')

Total of 65992 compilable code:
C/testcases/CWE761_Free_Pointer_Not_at_Start_of_Buffer/CWE761_Free_Pointer_Not_at_Start_of_Buffer__char_environment_54b.o

35551 of the compilable Juliet code is C:
C/testcases/CWE761_Free_Pointer_Not_at_Start_of_Buffer/CWE761_Free_Pointer_Not_at_Start_of_Buffer__char_environment_54b.c

30441 of the compilable Juliet code is C++:
C/testcases/CWE761_Free_Pointer_Not_at_Start_of_Buffer/CWE761_Free_Pointer_Not_at_Start_of_Buffer__char_fixed_string_82_goodB2G.cpp

35551 + 30441 = 65992 matching the total compilable code in the Juliet dataset




In [ ]:
#Generate the rest of the json ASTs from the Juliet Dataset 
for i in range(len(compilable_juliet_c_code)):
    subprocess.run(["python3", "multi_file_c_json.py", compilable_juliet_c_code[i]])
print('Done generating Juliet C json ASTs.  Verify with ls cmd')  

Done generating Juliet C json ASTs.  Verify with ls cmd


In [ ]:
#cd to testcases and use cmd ls to very presence of json ASTs
!ls

C      multi_file_c_json.py		    pycparser
drive  phase1_processed_juliet_dataset.zip  sample_data


In [ ]:
%cd C/testcases/

/content/C/testcases


In [ ]:
!ls

CWE114_Process_Control
CWE121_Stack_Based_Buffer_Overflow
CWE122_Heap_Based_Buffer_Overflow
CWE123_Write_What_Where_Condition
CWE124_Buffer_Underwrite
CWE126_Buffer_Overread
CWE127_Buffer_Underread
CWE134_Uncontrolled_Format_String
CWE15_External_Control_of_System_or_Configuration_Setting
CWE176_Improper_Handling_of_Unicode_Encoding
CWE188_Reliance_on_Data_Memory_Layout
CWE190_Integer_Overflow
CWE191_Integer_Underflow
CWE194_Unexpected_Sign_Extension
CWE195_Signed_to_Unsigned_Conversion_Error
CWE196_Unsigned_to_Signed_Conversion_Error
CWE197_Numeric_Truncation_Error
CWE222_Truncation_of_Security_Relevant_Information
CWE223_Omission_of_Security_Relevant_Information
CWE226_Sensitive_Information_Uncleared_Before_Release
CWE23_Relative_Path_Traversal
CWE242_Use_of_Inherently_Dangerous_Function
CWE244_Heap_Inspection
CWE247_Reliance_on_DNS_Lookups_in_Security_Decision
CWE252_Unchecked_Return_Value
CWE253_Incorrect_Check_of_Function_Return_Value
CWE256_Plaintext_Storage_of_Password
CWE259_Ha

In [ ]:
print(CWE_makefile_dir)

['C/testcases/CWE789_Uncontrolled_Mem_Alloc/s01/Makefile', 'C/testcases/CWE789_Uncontrolled_Mem_Alloc/s02/Makefile', 'C/testcases/CWE484_Omitted_Break_Statement_in_Switch/Makefile', 'C/testcases/CWE194_Unexpected_Sign_Extension/s01/Makefile', 'C/testcases/CWE194_Unexpected_Sign_Extension/s02/Makefile', 'C/testcases/CWE563_Unused_Variable/Makefile', 'C/testcases/CWE588_Attempt_to_Access_Child_of_Non_Structure_Pointer/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s03/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s01/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s04/Makefile', 'C/testcases/CWE78_OS_Command_Injection/s02/Makefile', 'C/testcases/CWE390_Error_Without_Action/Makefile', 'C/testcases/CWE364_Signal_Handler_Race_Condition/Makefile', 'C/testcases/CWE188_Reliance_on_Data_Memory_Layout/Makefile', 'C/testcases/CWE127_Buffer_Underread/s03/Makefile', 'C/testcases/CWE127_Buffer_Underread/s01/Makefile', 'C/testcases/CWE127_Buffer_Underread/s02/Makefile', 'C/testcases/CWE77

In [ ]:
%cd CWE484_Omitted_Break_Statement_in_Switch/

/content/C/testcases/CWE484_Omitted_Break_Statement_in_Switch


In [ ]:
!ls

compilation.log
CWE484.bat
CWE484_Omitted_Break_Statement_in_Switch__basic_01.c
CWE484_Omitted_Break_Statement_in_Switch__basic_01.json
CWE484_Omitted_Break_Statement_in_Switch__basic_01.o
CWE484_Omitted_Break_Statement_in_Switch__basic_02.c
CWE484_Omitted_Break_Statement_in_Switch__basic_02.json
CWE484_Omitted_Break_Statement_in_Switch__basic_02.o
CWE484_Omitted_Break_Statement_in_Switch__basic_03.c
CWE484_Omitted_Break_Statement_in_Switch__basic_03.json
CWE484_Omitted_Break_Statement_in_Switch__basic_03.o
CWE484_Omitted_Break_Statement_in_Switch__basic_04.c
CWE484_Omitted_Break_Statement_in_Switch__basic_04.json
CWE484_Omitted_Break_Statement_in_Switch__basic_04.o
CWE484_Omitted_Break_Statement_in_Switch__basic_05.c
CWE484_Omitted_Break_Statement_in_Switch__basic_05.json
CWE484_Omitted_Break_Statement_in_Switch__basic_05.o
CWE484_Omitted_Break_Statement_in_Switch__basic_06.c
CWE484_Omitted_Break_Statement_in_Switch__basic_06.json
CWE484_Omitted_Break_Statement_in_Switch__basic_06.o
C

In [ ]:
%cd ..

/content/C/testcases


In [ ]:
!ls

CWE114_Process_Control
CWE121_Stack_Based_Buffer_Overflow
CWE122_Heap_Based_Buffer_Overflow
CWE123_Write_What_Where_Condition
CWE124_Buffer_Underwrite
CWE126_Buffer_Overread
CWE127_Buffer_Underread
CWE134_Uncontrolled_Format_String
CWE15_External_Control_of_System_or_Configuration_Setting
CWE176_Improper_Handling_of_Unicode_Encoding
CWE188_Reliance_on_Data_Memory_Layout
CWE190_Integer_Overflow
CWE191_Integer_Underflow
CWE194_Unexpected_Sign_Extension
CWE195_Signed_to_Unsigned_Conversion_Error
CWE196_Unsigned_to_Signed_Conversion_Error
CWE197_Numeric_Truncation_Error
CWE222_Truncation_of_Security_Relevant_Information
CWE223_Omission_of_Security_Relevant_Information
CWE226_Sensitive_Information_Uncleared_Before_Release
CWE23_Relative_Path_Traversal
CWE242_Use_of_Inherently_Dangerous_Function
CWE244_Heap_Inspection
CWE247_Reliance_on_DNS_Lookups_in_Security_Decision
CWE252_Unchecked_Return_Value
CWE253_Incorrect_Check_of_Function_Return_Value
CWE256_Plaintext_Storage_of_Password
CWE259_Ha

In [ ]:
%cd CWE124_Buffer_Underwrite/

/content/C/testcases/CWE124_Buffer_Underwrite


In [ ]:
!ls

s01  s02  s03  s04


In [ ]:
%cd s01/

/content/C/testcases/CWE124_Buffer_Underwrite/s01


In [ ]:
!ls

compilation.log
CWE124_Buffer_Underwrite__char_alloca_cpy_01.c
CWE124_Buffer_Underwrite__char_alloca_cpy_01.json
CWE124_Buffer_Underwrite__char_alloca_cpy_01.o
CWE124_Buffer_Underwrite__char_alloca_cpy_02.c
CWE124_Buffer_Underwrite__char_alloca_cpy_02.json
CWE124_Buffer_Underwrite__char_alloca_cpy_02.o
CWE124_Buffer_Underwrite__char_alloca_cpy_03.c
CWE124_Buffer_Underwrite__char_alloca_cpy_03.json
CWE124_Buffer_Underwrite__char_alloca_cpy_03.o
CWE124_Buffer_Underwrite__char_alloca_cpy_04.c
CWE124_Buffer_Underwrite__char_alloca_cpy_04.json
CWE124_Buffer_Underwrite__char_alloca_cpy_04.o
CWE124_Buffer_Underwrite__char_alloca_cpy_05.c
CWE124_Buffer_Underwrite__char_alloca_cpy_05.json
CWE124_Buffer_Underwrite__char_alloca_cpy_05.o
CWE124_Buffer_Underwrite__char_alloca_cpy_06.c
CWE124_Buffer_Underwrite__char_alloca_cpy_06.json
CWE124_Buffer_Underwrite__char_alloca_cpy_06.o
CWE124_Buffer_Underwrite__char_alloca_cpy_07.c
CWE124_Buffer_Underwrite__char_alloca_cpy_07.json
CWE124_Buffer_Underwrit

In [ ]:
%cd ..

/content


In [ ]:
#create a list of the Juliet C Json ASTs
juliet_c_ast_jsons = []
juliet_json_paths = Path('./C/testcases').glob('**/*.json')
for path in juliet_json_paths:
  juliet_c_ast_jsons.append(str(path))
  # str because path is an object not string
print("{} jsons of C ASTs from the programming submissions:".format(len(juliet_c_ast_jsons)))
print(juliet_c_ast_jsons[0])
print('\n')   

35551 jsons of C ASTs from the programming submissions:
C/testcases/CWE761_Free_Pointer_Not_at_Start_of_Buffer/CWE761_Free_Pointer_Not_at_Start_of_Buffer__char_console_15.json




In [ ]:
with open('compilable_juliet_code_ls_file', 'wb') as fp:
  pickle.dump (compilable_juliet_code, fp)

In [ ]:
with open('compilable_juliet_c_code_ls_file', 'wb') as fp:
  pickle.dump (compilable_juliet_c_code, fp)

In [ ]:
with open('compilable_juliet_cpp_code_ls_file', 'wb') as fp:
  pickle.dump (compilable_juliet_cpp_code, fp)

In [ ]:
with open('juliet_c_ast_jsons_ls_file', 'wb') as fp:
  pickle.dump (juliet_c_ast_jsons, fp)

In [ ]:
!ls

C				    juliet_c_ast_jsons_ls_file
compilable_juliet_c_code_ls_file    multi_file_c_json.py
compilable_juliet_code_ls_file	    phase1_processed_juliet_dataset.zip
compilable_juliet_cpp_code_ls_file  pycparser
drive				    sample_data


    Curating the compiled files and metadata for further processing and use in  building models     

**3.3 Curating processed Programming Competition submissions** 
----

In [ ]:
#move pickled metadata file into folders for compression and curating for PHASE 2 of Big Data Cleaning Pipelne
%cp decodable_submisisons_ls_file submisisons_ls_file cpp_compiled_ls_file c_compiled_ls_file programming_competition_c_ast_jsons_ls_file c_json.py ProgramData/

In [ ]:
#Creating tar archive of processed Progamming competion dataset and metadata 
shutil.make_archive('phase1_programing_competition','tar','/content/','ProgramData')

'/content/phase1_programing_competition.tar'

In [ ]:
#Verify tarball of processed files
check_processed_archive = []
with tarfile.open('phase1_programing_competition.tar', 'r') as submissions:
  for member in submissions:
    if member.isdir() and member.name.count ('/') > 0:
     # The '/' > 0 is to ensure the root directory is ignored 
       print(member.name) 
    if member.isfile():
     check_processed_archive.append (member.name)
print("\n Number of files in tarball for export {0}".format(len(check_processed_archive)))

ProgramData/16
ProgramData/91
ProgramData/23
ProgramData/81
ProgramData/43
ProgramData/8
ProgramData/36
ProgramData/40
ProgramData/21
ProgramData/6
ProgramData/35
ProgramData/24
ProgramData/3
ProgramData/25
ProgramData/92
ProgramData/2
ProgramData/4
ProgramData/26
ProgramData/58
ProgramData/19
ProgramData/39
ProgramData/75
ProgramData/83
ProgramData/50
ProgramData/59
ProgramData/18
ProgramData/74
ProgramData/100
ProgramData/76
ProgramData/88
ProgramData/82
ProgramData/57
ProgramData/94
ProgramData/27
ProgramData/1
ProgramData/98
ProgramData/32
ProgramData/45
ProgramData/7
ProgramData/13
ProgramData/9
ProgramData/66
ProgramData/85
ProgramData/90
ProgramData/47
ProgramData/17
ProgramData/101
ProgramData/53
ProgramData/28
ProgramData/79
ProgramData/54
ProgramData/37
ProgramData/33
ProgramData/72
ProgramData/95
ProgramData/20
ProgramData/12
ProgramData/49
ProgramData/46
ProgramData/56
ProgramData/93
ProgramData/5
ProgramData/86
ProgramData/80
ProgramData/73
ProgramData/97
ProgramData/102
P

In [ ]:
!cp phase1_programing_competition.tar "/content/drive/My Drive/2020/Fall2020/individual_lab_members/ebenx007/"

**3.4 Curating processed juliet source code** 
----

In [ ]:
!ls

C				    juliet_c_ast_jsons_ls_file
compilable_juliet_c_code_ls_file    multi_file_c_json.py
compilable_juliet_code_ls_file	    phase1_processed_juliet_dataset.zip
compilable_juliet_cpp_code_ls_file  pycparser
drive				    sample_data


In [ ]:
#Copy of files processed on Dec 3, 2020 
%cp -r compilable_juliet_code_ls_file compilable_juliet_c_code_ls_file compilable_juliet_cpp_code_ls_file juliet_c_ast_jsons_ls_file multi_file_c_json.py pycparser/  /content/C/

In [ ]:
#NOT DONE move pickled metadata file into folders for compression and curating for PHASE 2 of Big Data Cleaning Pipelne NOT DONE
%cp juliet_dataset_CWE_testcases_paths_ls_file juliet_dataset_CWE_ls_file juliet_dataset_ls_file CWE_makefile_dir_ls_file /content/C/

In [ ]:
#zipping processed juliet dataset and metadata
shutil.make_archive('curated_processed_juliet_dataset','zip','/content/','C')

'/content/curated_processed_juliet_dataset.zip'

In [ ]:
#Verify zipped archive of processed juliet dataset
processed_juliet_dataset_ls = []
with zipfile.ZipFile('curated_processed_juliet_dataset.zip', 'r') as jz:
  for member in jz.namelist():
    if member.endswith('/') and member.count('/') > 2:
      # > 2  '/' count to ignore none testcases related directories in the archive
      print( member)  
    if (not member.endswith('/')) and member.count('/') > 2:
      # > 2 '/' count to append to list only files in testcases directories, ignoring files inthe root directory  
      processed_juliet_dataset_ls.append(member)
    processed_juliet_dataset_ls.append(member) 
print("Number of processed Juliet C/C++ test files and metadata {0}".format(len(processed_juliet_dataset_ls)))

C/testcases/CWE114_Process_Control/
C/testcases/CWE121_Stack_Based_Buffer_Overflow/
C/testcases/CWE122_Heap_Based_Buffer_Overflow/
C/testcases/CWE123_Write_What_Where_Condition/
C/testcases/CWE124_Buffer_Underwrite/
C/testcases/CWE126_Buffer_Overread/
C/testcases/CWE127_Buffer_Underread/
C/testcases/CWE134_Uncontrolled_Format_String/
C/testcases/CWE15_External_Control_of_System_or_Configuration_Setting/
C/testcases/CWE176_Improper_Handling_of_Unicode_Encoding/
C/testcases/CWE188_Reliance_on_Data_Memory_Layout/
C/testcases/CWE190_Integer_Overflow/
C/testcases/CWE191_Integer_Underflow/
C/testcases/CWE194_Unexpected_Sign_Extension/
C/testcases/CWE195_Signed_to_Unsigned_Conversion_Error/
C/testcases/CWE196_Unsigned_to_Signed_Conversion_Error/
C/testcases/CWE197_Numeric_Truncation_Error/
C/testcases/CWE222_Truncation_of_Security_Relevant_Information/
C/testcases/CWE223_Omission_of_Security_Relevant_Information/
C/testcases/CWE226_Sensitive_Information_Uncleared_Before_Release/
C/testcases/C

In [ ]:
!cp curated_processed_juliet_dataset.zip "/content/drive/My Drive/2020/Fall2020/individual_lab_members/ebenx007/"